In [ ]:
# %pip install plotly pandas
# %pip install nbformat>=4.2.0

In [ ]:
import pandas as pd

import plotly.express as px

# Read the file, skipping the first row (number of particles)
with open('../res/rt_is.123456789', 'r') as f:
    n_particles = int(f.readline())
    df = pd.read_csv(f, delim_whitespace=True, header=None,
                     names=['id', 'x', 'y', 'z', 'p1', 'p2', 'p3', 'p4'])

# Plot with Y axis upwards
fig = px.scatter_3d(df, x='x', y='y', z='z', color='id', title='3D Scatter of Particles')
fig.update_layout(scene=dict(yaxis=dict(title='Y (Upwards)')))
fig.show()

In [ ]:
# Filter rows where p1, p2, p3, and p4 are not -1
filtered_df = df[(df['p1'] != -1) & (df['p2'] != -1) & (df['p3'] != -1) & (df['p4'] != -1)]

fig = px.scatter_3d(filtered_df, x='x', y='y', z='z', color='id', title='3D Scatter of Particles (p1,p2,p3,p4 != -1)')
fig.update_layout(scene=dict(yaxis=dict(title='Y (Upwards)')))
fig.show()

In [ ]:
# Read the origin file with integer ID column
origin_df = pd.read_csv(
    '../origin/origin_00000.txt',
    comment='#',
    sep='\s+',
    header=None,
    names=['ID', 'X', 'Y', 'Z', 'xc0', 'yc0', 'xc1', 'yc1', 'xc2', 'yc2', 'xc3', 'yc3']
)
origin_df['ID'] = origin_df['ID'].astype(int)
origin_df.head()

In [ ]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# 3D scatter plot for X, Y, Z
fig_3d = go.Figure(data=[go.Scatter3d(
    x=origin_df['X'],
    y=origin_df['Y'],
    z=origin_df['Z'],
    mode='markers',
    marker=dict(size=2),
    name='X,Y,Z'
)])
fig_3d.update_layout(title='3D Scatter: X, Y, Z', scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'
))
fig_3d.show()

# 4 subplots for xc0/yc0, xc1/yc1, xc2/yc2, xc3/yc3
fig_2d = make_subplots(rows=2, cols=2, subplot_titles=['xc0 vs yc0', 'xc1 vs yc1', 'xc2 vs yc2', 'xc3 vs yc3'])

fig_2d.add_trace(go.Scatter(x=origin_df['xc0'], y=origin_df['yc0'], mode='markers', marker=dict(size=2), name='xc0/yc0'), row=1, col=1)
fig_2d.add_trace(go.Scatter(x=origin_df['xc1'], y=origin_df['yc1'], mode='markers', marker=dict(size=2), name='xc1/yc1'), row=1, col=2)
fig_2d.add_trace(go.Scatter(x=origin_df['xc2'], y=origin_df['yc2'], mode='markers', marker=dict(size=2), name='xc2/yc2'), row=2, col=1)
fig_2d.add_trace(go.Scatter(x=origin_df['xc3'], y=origin_df['yc3'], mode='markers', marker=dict(size=2), name='xc3/yc3'), row=2, col=2)

fig_2d.update_layout(height=700, width=900, title_text="Scatter Plots: xcN vs ycN")
fig_2d.update_xaxes(title_text="xc0", row=1, col=1)
fig_2d.update_yaxes(title_text="yc0", row=1, col=1)
fig_2d.update_xaxes(title_text="xc1", row=1, col=2)
fig_2d.update_yaxes(title_text="yc1", row=1, col=2)
fig_2d.update_xaxes(title_text="xc2", row=2, col=1)
fig_2d.update_yaxes(title_text="yc2", row=2, col=1)
fig_2d.update_xaxes(title_text="xc3", row=2, col=2)
fig_2d.update_yaxes(title_text="yc3", row=2, col=2)

fig_2d.show()

In [ ]:
import glob

# Get the 4 files in ascending order
file_list = sorted(glob.glob('../origin/origin_*.txt'))[:4]

# Read and concatenate all files
dfs = []
for file in file_list:
    df_part = pd.read_csv(
        file,
        comment='#',
        sep='\s+',
        header=None,
        names=['ID', 'X', 'Y', 'Z', 'xc0', 'yc0', 'xc1', 'yc1', 'xc2', 'yc2', 'xc3', 'yc3']
    )
    df_part['ID'] = df_part['ID'].astype(int)
    dfs.append(df_part)
all_df = pd.concat(dfs, ignore_index=True)

# Group by ID
grouped = all_df.groupby('ID')

# Get the first 10 unique IDs
first_10_ids = all_df['ID'].drop_duplicates().head(10)

import plotly.graph_objects as go

fig = go.Figure()
for pid in first_10_ids:
    traj = grouped.get_group(pid)
    fig.add_trace(go.Scatter3d(
        x=traj['X'],
        y=traj['Y'],
        z=traj['Z'],
        mode='lines+markers',
        name=f'ID {pid}'
    ))

fig.update_layout(
    title='Trajectories of First 10 Particles',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    height=700,
    width=900
)
fig.show()

In [ ]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Create a 2x2 subplot for xc0/yc0, xc1/yc1, xc2/yc2, xc3/yc3 for each frame
fig_frames = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "Frame 0: xc0 vs yc0", "Frame 1: xc1 vs yc1",
        "Frame 2: xc2 vs yc2", "Frame 3: xc3 vs yc3"
    ]
)
for pid in first_10_ids:
    for i in range(4):
        frame_df = dfs[i]
        particle = frame_df[frame_df['ID'] == pid]
        fig_frames.add_trace(
            go.Scatter(
                x=particle[f'xc{i}'],
                y=particle[f'yc{i}'],
                mode='markers',
                marker=dict(size=6),
                name=f'ID {pid} Frame {i}'
            ),
            row=(i // 2) + 1,
            col=(i % 2) + 1
        )

fig_frames.update_layout(
    height=700, width=900,
    title_text="Scatter Plots: xcN vs ycN for Each Frame"
)
fig_frames.update_xaxes(title_text="xc0", row=1, col=1)
fig_frames.update_yaxes(title_text="yc0", row=1, col=1)
fig_frames.update_xaxes(title_text="xc1", row=1, col=2)
fig_frames.update_yaxes(title_text="yc1", row=1, col=2)
fig_frames.update_xaxes(title_text="xc2", row=2, col=1)
fig_frames.update_yaxes(title_text="yc2", row=2, col=1)
fig_frames.update_xaxes(title_text="xc3", row=2, col=2)
fig_frames.update_yaxes(title_text="yc3", row=2, col=2)

fig_frames.show()

In [ ]:
import numpy as np

# Calculate displacement for each of the first 10 particles (by ID)
def displacement_3d(group):
    start = group.iloc[0][['X', 'Y', 'Z']].values
    end = group.iloc[-1][['X', 'Y', 'Z']].values
    return np.linalg.norm(end - start)

displacements = grouped.apply(displacement_3d).loc[first_10_ids]
displacements.name = 'displacement_3d'
displacements


In [ ]:
all_df.head()

In [ ]:
def write_targets_to_file(targets, filename):
    """
    Write target data to a file in the specified format.
    
    Parameters:
    targets (list): List of target objects with methods pnr(), pos(), count_pixels(), sum_grey_value(), and tnr().
    filename (str): The name of the file to write the data to.
    """
    num_targets = len(targets)
    
    # Create a numpy array from the target data
    target_arr = np.array(
                [([t.pnr(), *t.pos(), *t.count_pixels(), t.sum_grey_value(), t.tnr()]) for t in targets]
            )
    # Save the target array to file using savetxt
    np.savetxt(
        filename,
        target_arr,
        fmt="%4d %9.4f %9.4f %5d %5d %5d %5d %5d",
        header=f"{num_targets}",
        comments="",
    )

In [ ]:
origin_df = pd.read_csv(
    '../origin/origin_00000.txt',
    comment='#',
    sep='\s+',
    header=None,
    names=['ID', 'X', 'Y', 'Z', 'xc0', 'yc0', 'xc1', 'yc1', 'xc2', 'yc2', 'xc3', 'yc3']
)
origin_df['ID'] = origin_df['ID'].astype(int)
origin_df.head()

In [ ]:
class Target:
    def __init__(self, pnr, pos, count_pixels, sum_grey_value, tnr):
        self._pnr = pnr
        self._pos = pos
        self._count_pixels = count_pixels
        self._sum_grey_value = sum_grey_value
        self._tnr = tnr

    def pnr(self):
        return self._pnr

    def pos(self):
        return self._pos

    def count_pixels(self):
        # Return as a tuple of 3 ints (for compatibility with write_targets_to_file)
        return (self._count_pixels, self._count_pixels, self._count_pixels)

    def sum_grey_value(self):
        return self._sum_grey_value

    def tnr(self):
        return self._tnr


In [ ]:
def read_origin_file(filepath):
    """
    Read an origin file and return a DataFrame with appropriate column names and ID as int.
    """
    df = pd.read_csv(
        filepath,
        comment='#',
        sep='\s+',
        header=None,
        names=['ID', 'X', 'Y', 'Z', 'xc0', 'yc0', 'xc1', 'yc1', 'xc2', 'yc2', 'xc3', 'yc3']
    )
    df['ID'] = df['ID'].astype(int)
    return df

In [ ]:
def write_all_targets(origin_df, filename_prefix="origin_targets_cam"):
    """
    Write 4 target files, one for each camera, using the given origin_df.
    """
    for cam in range(4):
        targets = [
            Target(
                pnr=i,
                pos=(row[f'xc{cam}'], row[f'yc{cam}']),
                count_pixels=3,
                sum_grey_value=10,
                tnr=-1
            )
            for i, row in origin_df.iterrows()
        ]
        write_targets_to_file(targets, f"{filename_prefix}{cam}.txt")



In [ ]:
# Write target files for 30 frames for each camera using read_origin_file and write_targets_to_file

for frame_idx in range(30):
    origin_path = f"../origin/origin_{frame_idx:05d}.txt"
    print(f"Processing frame {frame_idx} from {origin_path}")
    origin_df = read_origin_file(origin_path)
    for cam in range(4):
        targets = [
            Target(
                pnr=i,
                pos=(row[f'xc{cam}'], row[f'yc{cam}']),
                count_pixels=3,
                sum_grey_value=10,
                tnr=-1
            )
            for i, row in origin_df.iterrows()
        ]
        out_path = f"../particle_images/c{cam}/c{cam}.{frame_idx:04d}_targets"
        print(f"Writing targets to {out_path}")
        write_targets_to_file(targets, out_path)